<a href="https://colab.research.google.com/github/yemanzhongting/MultiCity/blob/main/Bert%E5%BE%AE%E8%B0%83816.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 8.1 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486150 sha256=05f7a492da3a8b166d6a266184c45ad1efc661309b1cf91fa91a8b740d18984e
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [ ]:
from opencc import OpenCC
a = '我是中國人'
b = OpenCC('s2t').convert(a)
print(b)

我是中國人


In [ ]:
开始调试

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 73.5 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
     |████████████████████████████████| 636 kB 71.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import random
import torch
import matplotlib.pyplot as plt
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

SEED = 123
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 1e-2
EPSILON = 1e-8

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
import pandas as pd
df=pd.read_csv('/content/筛选500-20210818.csv',encoding='utf_8_sig')
df.head()

,xuhao,fenlei,leibie,title,juzi,label,Unnamed: 6
0,22,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,案晁公武《读书志》曰：王冰谓《灵枢》即《汉志·黄帝内经》十八卷之九，或谓好事者於皇甫谧所集《...,d,NaN
1,23,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,又李濂《医史》载元吕复《群经古方论》曰：《内经》，《灵枢》，汉、隋、唐志皆不录，隋有《针经》...,d,NaN
2,24,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,或谓王冰以《九灵》更名为《灵枢》，又谓《九灵》尤详於针，故皇甫谧名之为《针经》。,d,NaN
3,25,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,苟一经而二名，不应《唐志》别出《针经》十二卷，是《灵枢》不及《素问》之古，宋、元人已言之矣。,d,NaN
4,26,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,近时杭世骏《道古堂集》亦有《灵枢经跋》， 曰《七略》、《汉·艺文志·黄帝内经》十八篇，皇甫谧...,d,NaN


In [ ]:
df.head()

,xuhao,fenlei,leibie,title,juzi,label
0,100,医家类,●卷一百三·子部十三●医家类一,△《金匮要略论注》·二十四卷（通行本）,是书亦名《金匮玉函经》，乃晋高平王叔和所编次。,a
1,146,医家类,●卷一百三·子部十三●医家类一,△《肘後备急方》·八卷（浙江范懋柱家天一阁藏本）,是书初名《肘後卒救方》，梁陶宏景补其阙漏，得一百一首，为《肘後百一方》。,a
2,147,医家类,●卷一百三·子部十三●医家类一,△《肘後备急方》·八卷（浙江范懋柱家天一阁藏本）,金杨用道又取唐慎微证类本草诸方附於肘後随证之下，为《附广肘後方》。,a
3,219,医家类,●卷一百三·子部十三●医家类一,△《千金要方》·九十三卷（两淮马裕家藏本）,思邈尝谓人命至重，贵於千金，一方济之，德逾於此。,a
4,220,医家类,●卷一百三·子部十三●医家类一,△《千金要方》·九十三卷（两淮马裕家藏本）,故所著方书以千金名。,a


In [ ]:
sentence,label=df['juzi'].values.tolist(),df['label'].values.tolist()
# readfile('hotel/pos.txt'), readfile('hotel/neg.txt')

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("ethanyt/guwenbert-base")

model = AutoModel.from_pretrained("ethanyt/guwenbert-base")

Downloading:   0%|          | 0.00/519 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/93.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at ethanyt/guwenbert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', cache_dir="E:/transformer_file/")
print(sentence[2])
print(tokenizer.tokenize(sentence[2]))
print(tokenizer.encode(sentence[2]))
print(tokenizer.convert_ids_to_tokens(tokenizer.encode(sentence[2])))

或谓王冰以《九灵》更名为《灵枢》，又谓《九灵》尤详於针，故皇甫谧名之为《针经》。
['或', '谓', '王', '冰', '以', '《', '九', '灵', '》', '更', '名', '为', '《', '灵', '枢', '》', '，', '又', '谓', '《', '九', '灵', '》', '尤', '详', '於', '针', '，', '故', '皇', '甫', '谧', '名', '之', '为', '《', '针', '经', '》', '。']
[0, 171, 92, 45, 1421, 8, 184, 152, 521, 185, 363, 74, 9, 184, 521, 1450, 185, 5, 60, 92, 184, 152, 521, 185, 884, 824, 5812, 1828, 5, 40, 274, 1102, 3818, 74, 6, 9, 184, 1828, 124, 185, 4, 2]
['[CLS]', '或', '谓', '王', '冰', '以', '《', '九', '灵', '》', '更', '名', '为', '《', '灵', '枢', '》', '，', '又', '谓', '《', '九', '灵', '》', '尤', '详', '於', '针', '，', '故', '皇', '甫', '谧', '名', '之', '为', '《', '针', '经', '》', '。', '[SEP]']


In [ ]:
#将每一句转成数字（大于126做截断，小于126做PADDING，加上首尾两个标识，长度总共等于128）
def convert_text_to_token(tokenizer, sentence, limit_size=126):

    tokens = tokenizer.encode(sentence[:limit_size])  #直接截断
    if len(tokens) < limit_size + 2:                  #补齐（pad的索引号就是0）
        tokens.extend([0] * (limit_size + 2 - len(tokens)))
    return tokens

input_ids = [convert_text_to_token(tokenizer, sen) for sen in sentence]

input_tokens = torch.tensor(input_ids)
print(input_tokens.shape)                    #torch.Size([10000, 128])

torch.Size([5226, 128])


In [ ]:
#建立mask
def attention_masks(input_ids):
    atten_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        atten_masks.append(seq_mask)
    return atten_masks

atten_masks = attention_masks(input_ids)
attention_tokens = torch.tensor(atten_masks)

In [ ]:
set(label)

{'a', 'b', 'c', 'd', 'e', 'f'}

In [ ]:
map_class = {
    'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5}
df['类别'] = df['label'].map(map_class)

In [ ]:
import numpy as np
labels=df['类别'].values.tolist()
targets = np.array(labels).reshape(-1, 1)   #(10000, 1)
total_targets = torch.tensor(targets)

In [ ]:
total_targets.shape

torch.Size([5226, 1])

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_tokens, total_targets, random_state=66, test_size=0.1)
train_masks, test_masks, _, _ = train_test_split(attention_tokens, input_tokens, random_state=66, test_size=0.1)
print(train_inputs.shape, test_inputs.shape)      #torch.Size([8000, 128]) torch.Size([2000, 128])
print(train_masks.shape)                          #torch.Size([8000, 128])和train_inputs形状一样

print(train_inputs[0])
print(train_masks[0])

torch.Size([4703, 128]) torch.Size([523, 128])
torch.Size([4703, 128])
tensor([   0,   60,   12,  160,  117,  470,   15,   74,   16,    5, 2928, 6743,
        6726,  157,  791,  911, 4662,  106,    5,  263, 5821,  228,  640,    5,
        3464, 6621,  408, 1074,    4,    2,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
for i, (train, mask, label) in enumerate(train_dataloader):
     print(train.shape, mask.shape, label.shape)               #torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16, 1])
     break
print('len(train_dataloader)=', len(train_dataloader))        #500

torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16, 1])
len(train_dataloader)= 294


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("ethanyt/guwenbert-base")
model = AutoModel.from_pretrained("ethanyt/guwenbert-base")

Some weights of the model checkpoint at ethanyt/guwenbert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels = 2)     #num_labels表示2个分类，好评和差评
model = BertForSequenceClassification.from_pretrained("ethanyt/guwenbert-base", num_labels =6)  
device = torch.device("cuda")#"cuda" if torch.cuda.is_available() else 
model.to(device)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ethanyt/guwenbert-base were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.6.attention.self.key.bias', 'roberta.encoder.layer.5.attention.output.dense.weight', 'roberta.encoder.layer.6.intermediate.dense.bias', 'roberta.encoder.layer.8.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'lm_head.decoder.bias', 'roberta.encoder.layer.4.output.LayerNorm.weight', 'roberta.encoder.layer.8.attention.output.LayerNorm.bias', 'roberta.encoder.layer.9.attention.output.dense.weight', 'roberta.encoder.layer.11.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.self.query.weight', 'roberta.encoder.layer.9.output.dense.weight', 'roberta.encoder.layer.2.output.LayerN

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(23292, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [ ]:
# # model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels = 2)     #num_labels表示2个分类，好评和差评
# model = BertForSequenceClassification.from_pretrained("ethanyt/guwenbert-base", num_labels =6)  
# device = torch.device("cuda",0)#"cuda" if torch.cuda.is_available() else 
# model.to(device)

In [ ]:
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = EPSILON)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': WEIGHT_DECAY},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr = LEARNING_RATE, eps = EPSILON)

In [ ]:
def binary_acc(preds, labels):      #preds.shape=(16, 2) labels.shape=torch.Size([16, 1])
  # print('pred is')
  # print(preds)
  # print(labels)

#   tensor([[-0.6207, -0.5117,  0.2221,  0.7096,  0.5865,  0.1338],
#         [-0.6637, -0.1734,  0.0489,  0.5567,  0.4895,  0.1555],
#         [-0.7304,  0.1899,  0.0502,  0.1732,  0.5673, -0.0090],
#         [-0.6466, -0.4081,  0.0429,  0.7352,  0.5475,  0.1929],
#         [-0.5700, -0.9003,  0.3235,  1.0641,  0.5718,  0.1983],
#         [-0.5476, -0.8967,  0.2668,  1.1331,  0.6050,  0.1603],
#         [-0.7848,  0.9896,  0.5479, -0.6765,  0.0710, -0.4884],
#         [-0.7449,  0.3604, -0.0929,  0.1121,  0.5175,  0.0084],
#         [-0.5131, -0.0384, -0.1605,  0.5261,  0.6951, -0.0636],
#         [-0.5233, -0.8732,  0.2598,  1.1383,  0.6104,  0.1456],
#         [-0.7705,  0.7923,  0.7255, -0.5743,  0.0851, -0.5042],
#         [-0.4873, -0.9013,  0.2035,  1.1127,  0.6868,  0.1336],
#         [-0.7950,  0.9261,  0.5125, -0.6366,  0.0999, -0.4297],
#         [-0.7784,  1.2285, -0.1008, -0.4990,  0.1477, -0.3198],
#         [-0.5900,  0.3439, -0.2164,  0.1100,  0.8683, -0.2011],
#         [-0.5321, -0.9551,  0.4726,  0.9874,  0.5740,  0.1277]],
#        device='cuda:0')
# tensor([[5],
#         [5],
#         [2],
#         [5],
#         [2],
#         [3],
#         [2],
#         [5],
#         [0],
#         [5],
#         [2],
#         [4],
#         [2],
#         [1],
#         [4],
#         [2]], device='cuda:0')

  correct = torch.eq(torch.max(preds, dim=1)[1], labels.flatten()).float()      #eq里面的两个参数的shape=torch.Size([16])    
  acc = correct.sum().item() / len(correct)
  return acc

In [ ]:
import time
import datetime
def format_time(elapsed):    
    elapsed_rounded = int(round((elapsed)))    
    return str(datetime.timedelta(seconds=elapsed_rounded))   #返回 hh:mm:ss 形式的时间

In [ ]:
def train(model, optimizer):
    t0 = time.time()
    avg_loss, avg_acc = [],[]

    model.train()
    for step, batch in enumerate(train_dataloader):

        # 每隔40个batch 输出一下所用时间.
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device), batch[2].long().to(device)

        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss, logits = output[0], output[1]

        avg_loss.append(loss.item())

        acc = binary_acc(logits, b_labels)
        avg_acc.append(acc)

        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 1.0)      #大于1的梯度将其设为1.0, 以防梯度爆炸
        optimizer.step()              #更新模型参数
        scheduler.step()              #更新learning rate

    avg_acc = np.array(avg_acc).mean()
    avg_loss = np.array(avg_loss).mean()
    return avg_loss, avg_acc

In [ ]:
def evaluate(model):
    avg_acc = []
    model.eval()         #表示进入测试模式

    with torch.no_grad():
        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device), batch[2].long().to(device)

            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

            acc = binary_acc(output[0], b_labels)
            avg_acc.append(acc)
    avg_acc = np.array(avg_acc).mean()
    return avg_acc

In [ ]:
epochs = 10
# training steps 的数量: [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

# 设计 learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [29]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(23292, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [30]:
for epoch in range(6):
    train_loss, train_acc = train(model, optimizer)
    print('epoch={},训练准确率={}，损失={}'.format(epoch, train_acc, train_loss))
    test_acc = evaluate(model)
    print("epoch={},测试准确率={}".format(epoch, test_acc))

  Batch    40  of    294.    Elapsed: 0:00:18.
  Batch    80  of    294.    Elapsed: 0:00:36.
  Batch   120  of    294.    Elapsed: 0:00:54.
  Batch   160  of    294.    Elapsed: 0:01:12.
  Batch   200  of    294.    Elapsed: 0:01:30.
  Batch   240  of    294.    Elapsed: 0:01:48.
  Batch   280  of    294.    Elapsed: 0:02:06.
epoch=0,训练准确率=0.8702947845804989，损失=0.3887704238295555
epoch=0,测试准确率=0.684745179063361
  Batch    40  of    294.    Elapsed: 0:00:18.
  Batch    80  of    294.    Elapsed: 0:00:36.
  Batch   120  of    294.    Elapsed: 0:00:55.
  Batch   160  of    294.    Elapsed: 0:01:13.
  Batch   200  of    294.    Elapsed: 0:01:31.
  Batch   240  of    294.    Elapsed: 0:01:49.
  Batch   280  of    294.    Elapsed: 0:02:07.
epoch=1,训练准确率=0.8668934240362812，损失=0.39554604483755673
epoch=1,测试准确率=0.684745179063361
  Batch    40  of    294.    Elapsed: 0:00:18.
  Batch    80  of    294.    Elapsed: 0:00:36.
  Batch   120  of    294.    Elapsed: 0:00:55.
  Batch   160  of    294. 

KeyboardInterrupt: ignored

In [31]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(23292, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [ ]:
#定义两个数组
Loss_list = []
Accuracy_list = []

Loss_list.append(train_loss / (len(train_dataset)))
Accuracy_list.append(100 * train_acc / (len(train_dataset)))

#我这里迭代了200次，所以x的取值范围为(0，200)，然后再将每次相对应的准确率以及损失率附在x上
x1 = range(0, 200)
x2 = range(0, 200)
y1 = Accuracy_list
y2 = Loss_list
plt.subplot(2, 1, 1)
plt.plot(x1, y1, 'o-')
plt.title('Test accuracy vs. epoches')
plt.ylabel('Test accuracy')
plt.subplot(2, 1, 2)
plt.plot(x2, y2, '.-')
plt.xlabel('Test loss vs. epoches')
plt.ylabel('Test loss')
plt.show()
plt.savefig("accuracy_loss.jpg")

In [ ]:
torch.save({'epoch': epochID + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN,
                            'optimizer': optimizer.state_dict(),'alpha': loss.alpha, 'gamma': loss.gamma},
                           checkpoint_path + '/m-' + launchTimestamp + '-' + str("%.4f" % lossMIN) + '.pth.tar')

In [ ]:
atten_masks

In [32]:
def predict(sen):

    input_id = convert_text_to_token(tokenizer, sen)
    input_token =  torch.tensor(input_id).long().to(device)            #torch.Size([128])

    atten_mask = [float(i>0) for i in input_id]
    attention_token = torch.tensor(atten_mask).long().to(device)       #torch.Size([128])

    output = model(input_token.view(1, -1), token_type_ids=None, attention_mask=attention_token.view(1, -1))     #torch.Size([128])->torch.Size([1, 128])否则会报错
    # print(output[0])
    # tensor([[-0.1717, -3.0720, -1.0502,  2.4945,  2.9568,  0.8045]],
    #    device='cuda:0', grad_fn=<AddmmBackward>)

    return torch.max(output[0], dim=1)[1]

In [ ]:
tf=pd.read_csv('/content/结果-20210808.csv',encoding='utf_8_sig')

In [ ]:
tf.head()

,xuhao,fenlei,leibie,title,juzi,label
0,0,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,唐王冰注。,b
1,1,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,《汉书·艺文志》载《黄帝内经》十八篇，无《素问》之名。,d
2,2,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,後汉张机《伤寒论》引之，始称《素问》。,d
3,3,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,晋皇甫谧《甲乙经序》，称《针经》九卷，《素问》九卷，皆为《内经》，与《汉志》十八篇之数合，则...,d
4,4,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,故《隋书·经籍志》始著录也，然《隋志》所载只八卷，全元起所注已阙其第七。,"b,d"


In [33]:
def get_label(juzi):
  try:
    label = predict(juzi)
    if label==0:
      return 'a'
    if label==1:
      return 'b'
    if label==2:
      return 'c'
    if label==3:
      return 'd'
    if label==4:
      return 'e'
    if label==5:
      return 'f'
  except Exception as e:
    print(e)
    return ""

In [34]:
df.head()

,xuhao,fenlei,leibie,title,juzi,label,Unnamed: 6,类别
0,22,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,案晁公武《读书志》曰：王冰谓《灵枢》即《汉志·黄帝内经》十八卷之九，或谓好事者於皇甫谧所集《...,d,NaN,3
1,23,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,又李濂《医史》载元吕复《群经古方论》曰：《内经》，《灵枢》，汉、隋、唐志皆不录，隋有《针经》...,d,NaN,3
2,24,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,或谓王冰以《九灵》更名为《灵枢》，又谓《九灵》尤详於针，故皇甫谧名之为《针经》。,d,NaN,3
3,25,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,苟一经而二名，不应《唐志》别出《针经》十二卷，是《灵枢》不及《素问》之古，宋、元人已言之矣。,d,NaN,3
4,26,医家类,●卷一百三·子部十三●医家类一,△《灵枢经》·十二卷（大理寺卿陆锡熊家藏本）,近时杭世骏《道古堂集》亦有《灵枢经跋》， 曰《七略》、《汉·艺文志·黄帝内经》十八篇，皇甫谧...,d,NaN,3


In [42]:
get_label("案晁公武《读书志》曰：王冰谓《灵枢》即《汉志·黄帝内经》十八卷之九，或谓好事者於皇甫谧所集")

'd'

In [36]:
# 方式二：保存加载整个模型
# 保存
torch.save(model, 'pre')
# 加载
# model = torch.load(PATH)
# model.eval()


In [41]:
model = torch.load( 'pre')
# model.eval()

In [38]:
# 方式三：保存用于继续训练的checkpoint或者多个模型
# 保存
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            }, 'pre_check')
# # 加载
# checkpoint = torch.load(PATH)
# start_epoch=checkpoint['epoch']
# model.load_state_dict(checkpoint['model_state_dict'])
# # 测试时
# model.eval()
# # 或者训练时
# model.train()


In [40]:
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(23292, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [ ]:
tf['pre']=tf.apply(lambda x :get_label(x['juzi']),axis=1)
#  df.apply(lambda x: bd09towgs84(x['lng'],x['lat'])[0], axis=1)

流式输出内容被截断，只能显示最后 5000 行内容。
tensor([[ 1.0717, -1.4034, -1.7492,  0.3135,  4.7588, -1.1694]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.9843,  0.3459, -1.0030, -3.7748,  3.4916, -1.0540]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.8279, -2.5795, -1.4462, -0.5032,  3.5533,  0.2301]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.3545,  4.9408, -1.6277, -2.6885, -1.9425,  0.1257]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.0339,  4.6992, -2.1089, -2.6570, -2.0846,  0.5055]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.1335,  5.1731, -2.0085, -2.5353, -0.9640, -0.6864]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.7559, -2.6932,  2.5218, -0.5860,  2.3561, -2.0171]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.5490, -1.2217, -1.1541,  5.3560, -1.2110,  0.2977]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 1.4504,  1.6693, -3.6087,  0.0861, -0.5485,  0.9099

In [ ]:
tf.head()

,xuhao,fenlei,leibie,title,juzi,label,pre
0,0,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,唐王冰注。,b,b
1,1,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,《汉书·艺文志》载《黄帝内经》十八篇，无《素问》之名。,d,d
2,2,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,後汉张机《伤寒论》引之，始称《素问》。,d,d
3,3,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,晋皇甫谧《甲乙经序》，称《针经》九卷，《素问》九卷，皆为《内经》，与《汉志》十八篇之数合，则...,d,d
4,4,医家类,●卷一百三·子部十三●医家类一,△《黄帝素问》·二十四卷（内府藏本）,故《隋书·经籍志》始著录也，然《隋志》所载只八卷，全元起所注已阙其第七。,"b,d",d


In [ ]:
tf.to_csv('miaomiao.csv',index=None,encoding='utf_8_sig')